In [ ]:
import zipfile
file_name=("duth-cv-2019-2020-hw-4.zip")
with zipfile.ZipFile(file_name,'r') as zip:
             zip.extractall()
             print("done")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import models
 
# # Create the model
model = models.Sequential()
# # model.add(layers.Conv2D(64, (11,11), padding='valid',activation='relu',   input_shape=(128, 128, 3))) # νομιζω ειναι το καλυτερο μου
# # model.add(layers.Conv2D(64, (11,11),  padding='valid', activation='relu'))
# # model.add(layers.Conv2D(64, (11,11),padding='valid',   activation='relu'))
# # model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))
          
# # model.add(layers.Conv2D(128, (11,11), padding='valid', activation='relu')) 
# # model.add(layers.Conv2D(128, (11,11), padding='valid',  activation='relu')) 
# # model.add(layers.Conv2D(128, (11,11), padding='valid',  activation='relu')) 
# # model.add(layers.MaxPooling2D(pool_size=(2,2), strides=2))


model.add(tf.keras.layers.Conv2D(64,  (11,11), padding='same', activation='relu', input_shape=(128, 128, 3)))
#model.add(layers.Conv2D(64,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))

# model.add(layers.Conv2D(128,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(128,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(128,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))
# model.add(layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(256,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))
# model.add(layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
# model.add(layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(512,  (11,11), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.2))




         
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()


In [ ]:
import os
base_dir = os.getcwd()
base_dir = os.path.join(base_dir, r'duth-cv-2019-2020-hw-4\vehicles')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255 , rotation_range = 10 , width_shift_range = 0.1 , height_shift_range = 0.1 , shear_range = 0.15 , zoom_range = 0.1 , channel_shift_range = 10. , horizontal_flip = True)
val_datagen  = ImageDataGenerator(rescale=1./255, rotation_range = 10 , width_shift_range = 0.1 , height_shift_range = 0.1 , shear_range = 0.15 , zoom_range = 0.1 , channel_shift_range = 10. , horizontal_flip = True)

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=50,
                                                    class_mode='categorical',
                                                    # color_mode='grayscale',
                                                    target_size=(128,128),
                                                    shuffle=True)     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  val_datagen.flow_from_directory(validation_dir,
                                                        batch_size=50,
                                                        class_mode='categorical',
                                                        #  color_mode='grayscale',
                                                         target_size=(128,128)) 

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image

import os
import csv

model = tf.keras.models.load_model('small_last4.h5')
rowlist = [['Id', 'Category']]

for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames:
        
        path = os.path.join(dirname, filename)
        img = image.load_img(path, target_size=(128, 128), grayscale=False, interpolation='bilinear')
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        classes_pred = model.predict(x)
        cls_pred = np.argmax(classes_pred)
        rowlist.append([filename, cls_pred])
        #print(filename, cls_pred)
        with open('output.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(rowlist)
        

# 